In [0]:
# %run ../build_database

safegraph
idaho/censustract_pkmap.parquet
idaho/places.parquet
idaho/tract_table.parquet
idaho/spatial.parquet


In [0]:
import pyspark.sql.functions as F
# safegraph.write.parquet("/working_sg.parquet")

# Data Preprocessing

In [0]:
# create safegraph dataset for munging
safegraph = spark.sql("""
    SELECT places.location_name,
        patterns.placekey,
        patterns.date_range_start,
        patterns.date_range_end,
        patterns.popularity_by_day,
        patterns.visits_by_day,
        -- patterns.visitor_home_cbgs,
        patterns.visitor_home_aggregation,
        patterns.raw_visit_counts,
        patterns.normalized_visits_by_state_scaling,
        patterns.raw_visitor_counts
        -- tract_x_pk.tract,
        -- tract_x_pk.county
    FROM safegraph.patterns AS patterns
    LEFT JOIN safegraph.places AS places
        ON patterns.placekey = places.placekey
    -- LEFT JOIN safegraph.spatial AS spatial
    --     ON patterns.placekey = spatial.placekey
    -- LEFT JOIN safegraph.censustract_pkmap AS tract_x_pk
    --     ON patterns.placekey = tract_x_pk.placekey
""")

# display(safegraph)

In [0]:
# convert from str to timestamps, remove dupes, filter down to churches, extract Sunday data / state_sampling_rate
safegraph = safegraph\
    .withColumns({
        'date_range_start': F.to_timestamp(F.col('date_range_start'), format="yyyy-MM-dd'T'HH:mm:ssxxx"),
        'date_range_end': F.to_timestamp(F.col('date_range_end'), format="yyyy-MM-dd'T'HH:mm:ssxxx"),
        'is_lds': F.lower(F.col('location_name')).rlike('(jesus christ of la(tt|dd|t)er|church of jesus christ of|(^| )lds(\b| )|day saints)'),
        'raw_sunday_visits': F.col('popularity_by_day.Sunday'),
        'state_sampling_rate': F.col('normalized_visits_by_state_scaling') / F.col('raw_visit_counts'),
        'visits_by_sunday': F.array_remove(
        F.transform('visits_by_day', lambda visits, i: F.when(
                i % 7 ==\
                    (F.dayofmonth(F.col('date_range_start')) - F.dayofweek(F.col('date_range_start')) + 7) % 7 # index of first Sunday
                , visits
            ).otherwise(-99999)
        ),
        -99999
        )
    })\
    .filter(F.col('is_lds') == True)\
    .dropDuplicates(['placekey', 'date_range_start']) 
# display(safegraph)

In [0]:
# safegraph = safegraph.drop('location_name', 'is_lds', 'visitor_home_aggregation', 'visitor_home_cbgs')
safegraph = safegraph.drop('popularity_by_day', 'is_lds') #'normalized_visits_by_state_scaling', 'raw_visit_counts', 'date_range_end',

# Exploration - Scaling Visitor Counts by "state_sampling_rate"

The `scaled_visitor_counts` are 5x times larger than the weekly sunday visit counts (found in `scaled_visits_by_sunday` which is a proxy for church attendance). So this won't work.

In [0]:
visitor_scaling = safegraph.select(
    'location_name',
    'placekey',
    'date_range_start',
    'date_range_end',
    'visitor_home_aggregation',
    'state_sampling_rate',
    'raw_visit_counts',
    'normalized_visits_by_state_scaling',
    'raw_sunday_visits',
    (F.col('raw_sunday_visits') * F.col('state_sampling_rate')).alias('scaled_sunday_visits'),
    'raw_visitor_counts',
    'visits_by_sunday',
    F.transform('visits_by_sunday', lambda visits: F.floor(visits * F.col('state_sampling_rate'))).alias('scaled_visits_by_sunday'),

    # can’t scale visitors with state_sampling_rate, visitors make multiple visits which inflates visitor counts 
    (F.col('raw_visitor_counts') * F.col('state_sampling_rate')).alias('scaled_visitor_counts')
).limit(100)

display(visitor_scaling)

location_name,placekey,date_range_start,date_range_end,visitor_home_aggregation,state_sampling_rate,raw_visit_counts,normalized_visits_by_state_scaling,raw_sunday_visits,scaled_sunday_visits,raw_visitor_counts,visits_by_sunday,scaled_visits_by_sunday,scaled_visitor_counts
The Church of Jesus Christ of Latter day Saints,222-222@3x5-4n2-qmk,2019-02-01T08:00:00.000+0000,2019-03-01T08:00:00.000+0000,"Map(16055000200 -> 4, 16055001700 -> 4, 16055001002 -> 4, 16055001200 -> 5, 16055000401 -> 6, 16055000700 -> 7, 16055000302 -> 4, 16055000402 -> 9, 16055000800 -> 7, 16055001300 -> 4, 16055000500 -> 5)",15.979296772500657,149.0,2380.915219102598,57,910.8199160325374,75.0,"List(15, 13, 14, 15)","List(239, 207, 223, 239)",1198.4472579375492
The Church of Jesus Christ of Latter day Saints,222-222@3x5-4n2-qmk,2019-03-01T08:00:00.000+0000,2019-04-01T07:00:00.000+0000,"Map(16055000602 -> 4, 16055001700 -> 5, 16055001002 -> 4, 16055001200 -> 4, 16055000401 -> 4, 16055000700 -> 11, 16055000302 -> 4, 16055000800 -> 6, 16055001300 -> 5, 16055000500 -> 4)",16.056397816858706,133.0,2135.5009096422077,56,899.1582777440875,65.0,"List(15, 9, 12, 13, 7)","List(240, 144, 192, 208, 112)",1043.6658580958158
The Church of Jesus Christ of Latter day Saints,222-222@3x5-4n2-qmk,2019-04-01T07:00:00.000+0000,2019-05-01T07:00:00.000+0000,"Map(16055001700 -> 4, 16055000401 -> 5, 16017950800 -> 4, 16055001001 -> 4, 16055000700 -> 11, 16055000402 -> 5, 16055000800 -> 6, 16055001300 -> 4, 16055000500 -> 4, 16055000900 -> 4)",15.47719821414956,103.0,1594.1514160574047,36,557.1791357093841,57.0,"List(4, 9, 12, 11)","List(61, 139, 185, 170)",882.200298206525
The Church of Jesus Christ of Latter day Saints,222-222@3x5-4n2-qmk,2019-05-01T07:00:00.000+0000,2019-06-01T07:00:00.000+0000,"Map(16055000200 -> 4, 16055000602 -> 4, 16055001700 -> 4, 16055001002 -> 4, 16055000401 -> 4, 16055001001 -> 4, 16055000700 -> 10, 16055000302 -> 7, 16055000402 -> 4, 16055000800 -> 10, 16055001300 -> 4, 16055000500 -> 4)",14.834980389609333,126.0,1869.207529090776,64,949.4387449349973,68.0,"List(15, 12, 24, 13)","List(222, 178, 356, 192)",1008.7786664934347
The Church of Jesus Christ of Latter day Saints,222-222@3x5-4n2-qmk,2019-06-01T07:00:00.000+0000,2019-07-01T07:00:00.000+0000,"Map(30063001600 -> 4, 16055001700 -> 4, 53063012902 -> 4, 16055000700 -> 5, 16055000302 -> 4, 16055000402 -> 6, 49035101900 -> 4, 16055000800 -> 4, 16055001300 -> 4, 16055000500 -> 4)",15.347567122551162,126.0,1933.7934574414464,75,1151.0675341913372,62.0,"List(19, 21, 14, 14, 7)","List(291, 322, 214, 214, 107)",951.5491615981721
The Church of Jesus Christ of Latter day Saints,222-222@3x5-4n2-qmk,2019-07-01T07:00:00.000+0000,2019-08-01T07:00:00.000+0000,"Map(16055001500 -> 4, 16055001700 -> 4, 16055000401 -> 8, 16055000700 -> 7, 16055000302 -> 5, 16055000402 -> 4, 16055000800 -> 5, 16055001300 -> 4, 16055000500 -> 4)",16.40532143209921,88.0,1443.6682860247306,57,935.1033216296549,54.0,"List(16, 13, 15, 13)","List(262, 213, 246, 213)",885.8873573333574
The Church of Jesus Christ of Latter day Saints,222-222@3x5-4n2-qmk,2019-08-01T07:00:00.000+0000,2019-09-01T07:00:00.000+0000,"Map(16055000200 -> 4, 16055001700 -> 4, 16055002000 -> 4, 16055000401 -> 4, 16055000700 -> 9, 53063012701 -> 4, 16001010401 -> 4, 16055000402 -> 4, 16055000800 -> 6, 16055001300 -> 5, 16055000500 -> 4, 16055001400 -> 4)",15.35943805488297,88.0,1351.6305488297014,42,645.0963983050848,56.0,"List(12, 15, 5, 10)","List(184, 230, 76, 153)",860.1285310734463
The Church of Jesus Christ of Latter day Saints,222-222@3x5-4n2-qmk,2019-09-01T07:00:00.000+0000,2019-10-01T07:00:00.000+0000,"Map(16055001800 -> 7, 16055001002 -> 4, 16055001200 -> 4, 16055000401 -> 4, 16055001001 -> 4, 16055000700 -> 17, 53063010202 -> 4, 16055000302 -> 4, 53017950500 -> 4, 16055000402 -> 7, 16055000800 -> 10, 16055001300 -> 7, 16055000500 -> 4, 16055000900 -> 4)",14.197775809008673,145.0,2058.6774923062576,77,1093.2287372936678,79.0,"List(13, 22, 14, 13, 15)","List(184, 

# Exploration - Normalizing Sunday Visit Counts

We assume `avg_weekly_sunday_visits_scaled` is a pretty good proxy for active members (most people only go to church once on Sunday), then we can scale up the visitors in `visitor_home_aggregation`.

This could be unreliable though because the unscaled `avg_weekly_sunday_visits` are different from `reported_home_visitor_agg_total`.

In [0]:
def merge(acc, x):
    count = acc.count + 1
    sum = acc.sum + x
    return F.struct(count.alias("count"), sum.alias("sum"))

sunday_df = safegraph.select(
    'placekey',
    'date_range_start',
    'visitor_home_aggregation',
    F.aggregate( # sum
        F.map_values('visitor_home_aggregation'),
        F.lit(0),
        lambda acc, x: acc + x
    ).alias('reported_home_visitor_agg_total'),
    F.col('raw_visitor_counts').alias('raw_monthly_visitor_counts'),
    F.col('raw_sunday_visits').alias('raw_monthly_sunday_visits'),
    'visits_by_sunday',
    F.aggregate( # average
        'visits_by_sunday',
        F.struct(F.lit(0).alias("count"), F.lit(0.0).alias("sum")),
        merge,
        lambda acc: acc.sum / acc.count
    ).alias('avg_weekly_sunday_visits'),
    F.aggregate( # average
        F.transform('visits_by_sunday', lambda visits: F.floor(visits * F.col('state_sampling_rate'))),
        F.struct(F.lit(0).alias("count"), F.lit(0.0).alias("sum")),
        merge,
        lambda acc: acc.sum / acc.count
    ).alias('avg_weekly_sunday_visits_scaled') # proxy for active church-goers
)

display(sunday_df)

placekey,date_range_start,visitor_home_aggregation,reported_home_visitor_agg_total,raw_monthly_visitor_counts,raw_monthly_sunday_visits,visits_by_sunday,avg_weekly_sunday_visits,avg_weekly_sunday_visits_scaled
222-222@3x5-4n2-qmk,2019-02-01T08:00:00.000+0000,"Map(16055000200 -> 4, 16055001700 -> 4, 16055001002 -> 4, 16055001200 -> 5, 16055000401 -> 6, 16055000700 -> 7, 16055000302 -> 4, 16055000402 -> 9, 16055000800 -> 7, 16055001300 -> 4, 16055000500 -> 5)",59,75.0,57,"List(15, 13, 14, 15)",14.25,227.0
222-222@3x5-4n2-qmk,2019-03-01T08:00:00.000+0000,"Map(16055000602 -> 4, 16055001700 -> 5, 16055001002 -> 4, 16055001200 -> 4, 16055000401 -> 4, 16055000700 -> 11, 16055000302 -> 4, 16055000800 -> 6, 16055001300 -> 5, 16055000500 -> 4)",51,65.0,56,"List(15, 9, 12, 13, 7)",11.2,179.2
222-222@3x5-4n2-qmk,2019-04-01T07:00:00.000+0000,"Map(16055001700 -> 4, 16055000401 -> 5, 16017950800 -> 4, 16055001001 -> 4, 16055000700 -> 11, 16055000402 -> 5, 16055000800 -> 6, 16055001300 -> 4, 16055000500 -> 4, 16055000900 -> 4)",51,57.0,36,"List(4, 9, 12, 11)",9.0,138.75
222-222@3x5-4n2-qmk,2019-05-01T07:00:00.000+0000,"Map(16055000200 -> 4, 16055000602 -> 4, 16055001700 -> 4, 16055001002 -> 4, 16055000401 -> 4, 16055001001 -> 4, 16055000700 -> 10, 16055000302 -> 7, 16055000402 -> 4, 16055000800 -> 10, 16055001300 -> 4, 16055000500 -> 4)",63,68.0,64,"List(15, 12, 24, 13)",16.0,237.0
222-222@3x5-4n2-qmk,2019-06-01T07:00:00.000+0000,"Map(30063001600 -> 4, 16055001700 -> 4, 53063012902 -> 4, 16055000700 -> 5, 16055000302 -> 4, 16055000402 -> 6, 49035101900 -> 4, 16055000800 -> 4, 16055001300 -> 4, 16055000500 -> 4)",43,62.0,75,"List(19, 21, 14, 14, 7)",15.0,229.6
222-222@3x5-4n2-qmk,2019-07-01T07:00:00.000+0000,"Map(16055001500 -> 4, 16055001700 -> 4, 16055000401 -> 8, 16055000700 -> 7, 16055000302 -> 5, 16055000402 -> 4, 16055000800 -> 5, 16055001300 -> 4, 16055000500 -> 4)",45,54.0,57,"List(16, 13, 15, 13)",14.25,233.5
222-222@3x5-4n2-qmk,2019-08-01T07:00:00.000+0000,"Map(16055000200 -> 4, 16055001700 -> 4, 16055002000 -> 4, 16055000401 -> 4, 16055000700 -> 9, 53063012701 -> 4, 16001010401 -> 4, 16055000402 -> 4, 16055000800 -> 6, 16055001300 -> 5, 16055000500 -> 4, 16055001400 -> 4)",56,56.0,42,"List(12, 15, 5, 10)",10.5,160.75
222-222@3x5-4n2-qmk,2019-09-01T07:00:00.000+0000,"Map(16055001800 -> 7, 16055001002 -> 4, 16055001200 -> 4, 16055000401 -> 4, 16055001001 -> 4, 16055000700 -> 17, 53063010202 -> 4, 16055000302 -> 4, 53017950500 -> 4, 16055000402 -> 7, 16055000800 -> 10, 16055001300 -> 7, 16055000500 -> 4, 16055000900 -> 4)",84,79.0,77,"List(13, 22, 14, 13, 15)",15.4,218.0
222-222@3x5-4n2-qmk,2019-10-01T07:00:00.000+0000,"Map(16055001500 -> 4, 16055001700 -> 4, 16055001800 -> 4, 16055001002 -> 4, 16055001900 -> 4, 16055000700 -> 17, 16055000302 -> 4, 38053962300 -> 4, 51760061000 -> 4, 16055000800 -> 9, 16055001300 -> 4, 16055000500 -> 4, 16055000900 -> 4)",70,71.0,50,"List(0, 9, 25, 16)",12.5,191.5
222-222@3x5-4n2-qmk,2019-11-01T07:00:00.000+0000,"Map(16055000200 -> 4, 53075000900 -> 4, 16055001700 -> 4, 16055001002 -> 4, 16055000401 -> 4, 16055001900 -> 4, 16055001001 -> 4, 16055000700 -> 10, 16043970300 -> 7, 16055000302 -> 4, 16055000402 -> 5, 16055000800 -> 7, 16055001300 -> 5, 16055000500 -> 4, 16055001400 -> 4, 16055000900 -> 4)",78,73.0,76,"List(19, 15, 19, 23)",19.0,328.75


# Creating the Target by Tract

1. I assume the relative proportion of visitors from each tract is accurately represented by the Safegraph data (realizing that we are missing some visitor home locations, the `reported_home_visitor_agg_total` is used in place of `raw_visitor_counts`).
2. I scale the count of visitors from each tract to reflect the total sacrament attendance estimated by `avg_weekly_sunday_visits_scaled`.
   - *It may be more effective to try and reduce the `avg_weekly_sunday_visits_scaled` to more accurately reflect count of visitor whose home data we actually have.*
3. Then we take the median `scaled_visitor_count` by place key.
4. Finally we sum up those visitor counts into the `target` by `tract`. We sum because most people don't attend two churches, and if they do they are more righteous which will inflate active member counts geospatially increasing the correlation produced by the proposed target `church_activity * distance_to_a_temple` and the likelihood for a temple to be announced in an area.

In [0]:
# steps 1, 2
tract_target = sunday_df.select(
    'placekey',
    'date_range_start',
    F.explode(
            F.transform_values(
            'visitor_home_aggregation',
            lambda _, visits: F.round(visits * (F.col('avg_weekly_sunday_visits_scaled') / F.col('reported_home_visitor_agg_total')))
        )#.alias("scaled_visitor_home_aggregation")
    ).alias('tract', 'scaled_visitor_count')

    # # checking to make sure the summed total is close to the avg_weekly_sunday_visits_scaled estimate
    # , F.aggregate( # sum
    #     F.map_values(F.transform_values('visitor_home_aggregation', lambda _, visits: F.round(visits * (F.col('avg_weekly_sunday_visits_scaled') / F.col('reported_home_visitor_agg_total'))))),
    #     F.lit(0.0),
    #     lambda acc, x: acc + x
    # ).alias("reported_home_visitor_agg_scaled_total"),
)

# display(tract_target.limit(100))

In [0]:
from pyspark.sql.window import Window

# step 3
tract_target = tract_target\
    .groupBy("tract", "placekey")\
    .agg(F.median("scaled_visitor_count"))

# display(tract_target.limit(100))


# # v2 - that doesn't work using window
#     .select(
#         "tract",
#         "placekey",
#         F.median(F.col("scaled_visitor_count")).over(Window.partitionBy("placekey", "tract")).alias('target')
#     )

In [0]:
# step 4
tract_target = tract_target\
    .groupBy("tract")\
    .sum('median(scaled_visitor_count)')\
    .withColumnRenamed("sum(median(scaled_visitor_count))", "target")

In [0]:
display(tract_target.limit(100))

tract,target
04013082019,20.0
04019004058,27.0
06085502500,0.0
16029960100,821.5
30111000704,17.0
36061011900,17.0
37051003104,12.0
49035113406,10.5
49049000801,59.0
49049010304,29.0
